In [9]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from arch import arch_model
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")

In [4]:
start_date = "2022-08-30"
end_date = "2024-09-30"

In [6]:
exchange_tickers = ["ZAR=X", "AUDUSD=X", "GBPUSD=X"]
exchange_data = {ticker: yf.download(ticker, start=start_date, end=end_date) for ticker in exchange_tickers}

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [8]:
for ticker, data in exchange_data.items():
    print(f"Data for {ticker}")
    print(data.head())

Data for ZAR=X
                 Open       High        Low      Close  Adj Close  Volume
Date                                                                     
2022-08-30  16.809641  16.972799  16.745100  16.809641  16.809641       0
2022-08-31  16.957800  17.087179  16.858629  16.957800  16.957800       0
2022-09-01  17.105209  17.304930  17.088301  17.105209  17.105209       0
2022-09-02  17.251900  17.345699  17.172701  17.251900  17.251900       0
2022-09-05  17.300900  17.418039  17.146500  17.300900  17.300900       0
Data for AUDUSD=X
                Open      High       Low     Close  Adj Close  Volume
Date                                                                 
2022-08-30  0.690751  0.695797  0.685990  0.690751   0.690751       0
2022-08-31  0.685740  0.690422  0.684340  0.685740   0.685740       0
2022-09-01  0.682850  0.684360  0.677172  0.682850   0.682850       0
2022-09-02  0.679530  0.685380  0.678109  0.679530   0.679530       0
2022-09-05  0.679030  0.68041

In [10]:
def calculate_returns(df):
    df['Returns'] = df['Adj Close'].pct_change().dropna() * 100
    return df['Returns'].dropna()

In [11]:
def fit_garch_model(returns, model_type='GARCH'):
    if model_type == 'ARCH':
        model = arch_model(returns, vol='Arch', p=1)
    elif model_type == 'GARCH':
        model = arch_model(returns, vol='Garch', p=1, q=1)
    elif model_type == 'EGARCH':
        model = arch_model(returns, vol='EGarch', p=1, q=1)
    elif model_type == 'TGARCH':
        model = arch_model(returns, vol='Garch', p=1, o=1, q=1)
    else:
        raise ValueError(f"Unsupport model type: {model_type}")
    
    model_fit = model.fit(disp='off')
    return model_fit

In [ ]:
def visualize_results(returns, model_results, title):

    # plot returns
    plt.figure(figsize=(20, 10))
    plt.subplot(2, 1, 1)
    plt.plot(returns, label='Returns', color='blue')
    plt.title(f"Daily Returns for {title}")
    plt.xlabel('Date')
    plt.ylabel('Returns')
    plt.legend()

    # plot volatility
    plt.subplot(2, 1, 2)